In [ ]:
#ini-iPython

# Executando no diretório principal

In [ ]:
import os
os.chdir('../../') #Executando na pasta Visual_Memory
import sys
sys.path.append('./include')
sys.path.append('./src')
sys.path.append('./Workbench/iPython')
#end-iPython

---- Imports ----

In [ ]:
# Libraries to be used.
import sys
#des-iPython sys.path.append("../include")
#des-iPython sys.path.append("../src")

In [ ]:
# The standard libraries used in the visual memory system.


In [ ]:
# Used class developed by RoboFEI-HT.
#ini-iPython - Usando Classe KalmanFilter como instanciação 
os.system("sed -i '/@abstractmethod/d' ./Workbench/iPython/KalmanFilter.py")
os.system("sed -i '/__metaclass__/d' ./Workbench/iPython/KalmanFilter.py")
#end-iPython
from KalmanFilter import * # Class responsible for implementing Kalman filter methods.
sym.init_printing() #debug-iPython

# Class to Landmark
Class responsible for performing landmarks tracking.
doxygen-iPython

In [ ]:
#des-iPython class Landmark(KalmanFilter):
#ini-tab -> Iniciando código classe

---- Variables ----

In [ ]:
#ini-iPython

# ------- Lista de funções -------

#self-iPython update
#self-iPython reset
#self-iPython predict
#self-iPython end

#end-iPython - Fim listas

# reset
.
doxygen-iPython

In [ ]:
#eself-iPython reset
#edes-iPython def reset(self):
def reset( ):
    #edes-iPython super(Landmark, self)._reset( )
    self._reset( )
    
    self._B = sym.Matrix([
        [0, 0, 0, 0, self._px],
        [0, 0, 0, 0, self._py],
        [0, 0, 0, 0, self._vx],
        [0, 0, 0, 0, self._vy],
        [0, 0, 0, 0, self._ax],
        [0, 0, 0, 0, self._ay],
    ])
    
    self._R = sym.zeros(6)
    #self-iPython reset

In [ ]:
#ini-iPython

## Teste reset

In [ ]:
s = Speeds( ) #debug-iPython
self = KalmanFilter(s, "Landmarks")
reset( )
self._B*sym.Matrix([
    [2.3],
    [1.9],
    [7.8],
    [1.5],
    [1],
])

In [ ]:
del self
del s
#end-iPython

# Constructor Class
doxygen-iPython

In [ ]:
s = Speeds( ) #debug-iPython
#des-iPython def __init__(self, a, s):
#ini-tab -> Criando construtor
# Instantiating constructor for inherited class.
#edes-iPython super(Landmark, self).__init__(a, s, "Landmarks")
self = KalmanFilter(None, s, "Landmarks")

# Creating characteristic variables for landmarks and reading.
#eself-iPython update
self._parameters.update({
    "linear_acceleration": True,
    "precision": 2,
}) #self-iPython update

self._parameters = self._conf.readVariables(self._parameters)

reset( )

In [ ]:
#end-tab -> Finalizando construtor

In [ ]:
#ini-iPython

## Teste construtor

In [ ]:
self._end( )
print self._parameters

vrx, vry, arx, ary = sym.symbols("vr_x vr_y ar_x ar_y")

__u = sym.Matrix([
    [vrx],
    [vry],
    [arx],
    [ary],
    [1],
])

self._B*__u

s[1]
#end-iPython

# update
.
doxygen-iPython

In [ ]:
#eself-iPython update
#edes-iPython def update(self, data):
def update(data):
    self._predictedstate["x"][2:, 0] = -self._speeds[data["movement"]]["x_speed"][:len(self._speeds[data["movement"]]["x_speed"])-1, 0]
    self._predictedstate["covariance"] = self._speeds[data["movement"]]["R"]
    
    #edes-iPython super(Landmark, self)._update(data)
    self._update(data)
    
    # Updating to current instant in time
    if self._predictedstate["time"] != time.time( ):
        #edes-iPython predict(tnow = None, movements = 1)
        self.predict(tnow = None, movements = 1)
    else:
        if self._args.savedata == True:
            try:
                a = np.load("./Data/Land.npy")
                a = np.concatenate((a,
                    [[
                        self._state["time"],
                        [float(i) for i in self._state["x"][:,0]],
                        [self._state["covariance"][i,i] for i in xrange(self._state["covariance"].shape[0])],
                        [float(i) for i in self._state["covariance"][:,0]],
                    ]]),
                    axis=0
                )
            except IOError:
                a = [[
                    self._state["time"],
                    [float(i) for i in self._state["x"][:,0]],
                    [self._state["covariance"][i,i] for i in xrange(self._state["covariance"].shape[0])],
                    [float(i) for i in self._state["covariance"][:,0]],
                ]]
            np.save("./Data/Land", a)

        if self._predictedstate["covariance"][0, 0] < self._parameters['vision_error']*self._parameters['precision'] and self._predictedstate["covariance"][1, 1] < self._parameters['precision']:
            self._bkb.write_float("VISUAL_MEMORY_LAND_X", self._predictedstate["x"][0,0])
            self._bkb.write_float("VISUAL_MEMORY_LAND_Y", self._predictedstate["x"][1,0])
            self._bkb.write_float("VISUAL_MEMORY_LAND_LOC", 1)
        else:        
            self._bkb.write_float("VISUAL_MEMORY_LAND_LOC", 0)
    
    return [data["movement"], self._predictedstate["x"], self._predictedstate["covariance"]] #self-iPython update

In [ ]:
#ini-iPython

## Teste update

In [ ]:
data = {
    "movement": 1,
    "target": 1,
    "pos": [0, 0],
    "time": 1.0,
}

print update(data)
#end-iPython

# predict
.
doxygen-iPython

In [ ]:
#eself-iPython predict
#edes-iPython def predict(self, tnow = None, movements = None):
def predict(tnow = None, movements = None):
    self._predictedstate["x"][2:, 0] = -self._speeds[movements]["x_speed"][:len(self._speeds[movements]["x_speed"])-1, 0]
    self._predictedstate["covariance"] = self._speeds[movements]["R"]
    
    #edes-iPython super(Landmark, self)._predict(tnow, movements)
    self._predict(tnow, movements) #self-iPython predict
    
    if self._args.savedata == True:
        try:
            a = np.load("./Data/Land.npy")
            a = np.concatenate((a,
                [[
                    self._state["time"],
                    [float(i) for i in self._state["x"][:,0]],
                    [self._state["covariance"][i,i] for i in xrange(self._state["covariance"].shape[0])],
                    [float(i) for i in self._state["covariance"][:,0]],
                ]]),
                axis=0
            )
        except IOError:
            a = [[
                self._state["time"],
                [float(i) for i in self._state["x"][:,0]],
                [self._state["covariance"][i,i] for i in xrange(self._state["covariance"].shape[0])],
                [float(i) for i in self._state["covariance"][:,0]],
            ]]
        np.save("./Data/Land", a)

    if self._predictedstate["covariance"][0, 0] < self._parameters['vision_error']*self._parameters['precision'] and self._predictedstate["covariance"][1, 1] < self._parameters['precision']:
        self._bkb.write_float("VISUAL_MEMORY_LAND_X", self._predictedstate["x"][0,0])
        self._bkb.write_float("VISUAL_MEMORY_LAND_Y", self._predictedstate["x"][1,0])
        self._bkb.write_float("VISUAL_MEMORY_LAND_LOC", 1)
    else:        
        self._bkb.write_float("VISUAL_MEMORY_LAND_LOC", 0)

# end
.
doxygen-iPython

In [ ]:
#eself-iPython end
#edes-iPython def end(self):
def end( ):
    self._end( ) #self-iPython end

In [ ]:
#ini-iPython

## Teste end

In [ ]:
end( )
#end-iPython

In [ ]:
#end-tab -> Finalizando classe

In [ ]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [ ]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
.
doxygen-iPython

In [ ]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [ ]:
#ini-iPython

## Teste nova_funcao

In [ ]:

#end-iPython

In [ ]:
#end-iPython